In [2]:
import os
import glob
import logging
import shutil
import sys

# --- CONFIGURACIÓN RUTAS ---
BASE_INPUT_DIR = "MultiViewVisibleThermalImagesHPE/test"
BASE_OUTPUT_DIR = "MultiViewVisibleImagesHPE_100"
TEST_OUTPUT_DIR = os.path.join(BASE_OUTPUT_DIR, "test")
SUB_DIRS = ["00_15", "00_16", "00_17"]

# Configuración básica de logs
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler(sys.stdout)]
)
logger = logging.getLogger(__name__)

def get_base_name(filename):
    """Extrae el identificador común (ej: rgb_0001.jpg) eliminando el prefijo de carpeta."""
    parts = filename.split('_')
    return "_".join(parts[2:]) if len(parts) > 2 else filename

def main():
    try:
        # 1. Escanear archivos en cada subcarpeta
        logger.info("🔍 Escaneando archivos para sincronización...")
        files_per_folder = {}
        
        for sd in SUB_DIRS:
            path = os.path.join(BASE_INPUT_DIR, sd)
            all_f = []
            for ext in ("*.jpg", "*.png", "*.jpeg"):
                all_f.extend(glob.glob(os.path.join(path, ext)))
            
            # Filtrar solo archivos que contienen "rgb" y mapear {nombre_base: nombre_completo}
            rgb_files = {get_base_name(os.path.basename(f)): os.path.basename(f) for f in all_f if "rgb" in os.path.basename(f).lower()}
            files_per_folder[sd] = rgb_files

        # 2. Encontrar la intersección (archivos presentes en las 3 carpetas)
        common_bases = sorted(list(
            set(files_per_folder["00_15"].keys()) & 
            set(files_per_folder["00_16"].keys()) & 
            set(files_per_folder["00_17"].keys())
        ))[:100]

        if len(common_bases) < 100:
            logger.warning(f"⚠️ Solo se han encontrado {len(common_bases)} archivos comunes.")
        else:
            logger.info(f"✅ Se han identificado las 100 imágenes comunes.")

        # 3. Copiar archivos a la carpeta de salida
        logger.info(f"📁 Copiando archivos a {TEST_OUTPUT_DIR}...")
        
        count = 0
        for sd in SUB_DIRS:
            dest_path = os.path.join(TEST_OUTPUT_DIR, sd)
            os.makedirs(dest_path, exist_ok=True)
            
            for base in common_bases:
                filename = files_per_folder[sd][base]
                src = os.path.join(BASE_INPUT_DIR, sd, filename)
                dst = os.path.join(dest_path, filename)
                
                shutil.copy2(src, dst)
                count += 1

        logger.info(f"✨ ¡Listo! Se han copiado {count} imágenes con éxito.")

    except Exception as e:
        logger.error(f"❌ Error durante el proceso: {e}")

if __name__ == "__main__":
    main()

2026-01-27 10:26:30,928 - INFO - 🔍 Escaneando archivos para sincronización...
2026-01-27 10:26:31,026 - INFO - ✅ Se han identificado las 100 imágenes comunes.
2026-01-27 10:26:31,027 - INFO - 📁 Copiando archivos a MultiViewVisibleImagesHPE_100/test...
2026-01-27 10:26:43,319 - INFO - ✨ ¡Listo! Se han copiado 300 imágenes con éxito.
